<a href="https://colab.research.google.com/github/souvikmajumder26/Any-Face-Clustering/blob/main/*Any_Face_Clustering_Streamlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install face_recognition
!pip install -q streamlit
# If ipykernel & ipython version related warnings are shown while installing streamlit: IGNORE

In [ ]:
%%writefile streamlit-app.py

import cv2
import face_recognition

import os
import time
import shutil
import tempfile
import numpy as np
import streamlit as st

from sklearn.cluster import DBSCAN
from imutils import build_montages
#from zipfile import ZipFile

flag1 = 0

st.set_page_config(layout = 'wide')

st.title("Any Face Clustering")

uploaded_files = st.file_uploader("Upload Images of Multiple Faces (with AT LEAST 3 images of a particular face):", type = ["png","jpg","jpeg"], accept_multiple_files = True)

no_of_files = len(uploaded_files)

if no_of_files>0:
  placeholder = st.empty()
  placeholder.success("{} Images uploaded successfully!".format(no_of_files))
  time.sleep(3)
  placeholder.empty()
  data = []
  
  for f in uploaded_files:
    tfile = tempfile.NamedTemporaryFile(delete = False)
    tfile.write(f.read())
    image = cv2.imread(tfile.name)
    rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    #st.write(rgb)
    #st.image(rgb)
    boxes = face_recognition.face_locations(rgb, model = "cnn")
    encodings = face_recognition.face_encodings(rgb, boxes)
    #st.write(boxes)
    #st.write(encodings)
    d = [{"imagePath": tfile.name, "loc": box, "encoding": enc} for (box, enc) in zip(boxes, encodings)]
    data.extend(d)
  
  # converting the data into a numpy array
  data_arr = np.array(data)
  # extracting the 128-d facial encodings and placing them in a list
  encodings_arr = [item["encoding"] for item in data_arr]

  # initializing and fitting the clustering model on the encoded data
  cluster = DBSCAN(min_samples = 3)
  cluster.fit(encodings_arr)
  st.balloons()
  #st.write(data)
  #st.write(data_arr)
  #st.write(np.shape(encodings_arr))
  #st.write(encodings_arr)  
  #st.write("Model Fit: SUCCESS")
  #st.write(cluster.n_features_in_)
  #st.write(cluster.labels_)
  
  labelIDs = np.unique(cluster.labels_)
  numUniqueFaces = len(np.where(labelIDs > -1)[0])
  
  st.subheader("Number of unique faces identified (excluding the unknown faces) is: " + str(numUniqueFaces))
  #st.write(labelIDs)

  if flag1 == 0:
    cols1 = st.columns(numUniqueFaces + 1)
    flag1 = 1
  
  #cols2 = st.columns(2)

  # loop over the unique face integers
  for labelID in labelIDs:
    # find all indexes into the 'pkl_data' array that belong to the current label ID, then randomly sample a maximum of 15 indexes from the set
    #print("[INFO] faces for face ID: {}".format(labelID))
    idxs = np.where(cluster.labels_ == labelID)[0]
    idxs = np.random.choice(idxs, size = min(15, len(idxs)), replace = False)
    # initialize the list of faces to include in the montage
    faces = []
    whole_images = []
    
    if labelID != -1:
      dir_name = 'face#{}'.format(labelID + 1)
      os.mkdir(dir_name)

    for i in idxs:
      # load the input image and extract the face ROI
      current_image = cv2.imread(data_arr[i]["imagePath"])
      rgb_current_image = cv2.cvtColor(current_image, cv2.COLOR_BGR2RGB)
      (top, right, bottom, left) = data_arr[i]["loc"]
      current_face = rgb_current_image[top:bottom, left:right]
      # force resize the face ROI to 96x96 and then add it to the faces montage list
      current_face = cv2.resize(current_face, (96, 96))
      whole_images.append(rgb_current_image)
      faces.append(current_face)

      if labelID != -1:
        face_image_name = 'image{}.jpg'.format(i)
        cv2.imwrite(os.path.join(dir_name, face_image_name), current_image)
    
    if labelID != -1:
      shutil.make_archive('zip_face#{}'.format(labelID + 1), 'zip', dir_name)
      # deleting the directory and image files contained in it
      shutil.rmtree('face#{}'.format(labelID + 1))
      
    # create a montage using 96x96 "tiles" with 5 rows and 5 columns
    montage = build_montages(faces, (96, 96), (3, 3))[0]
    
    #if labelID != -1:
      #zip_obj = ZipFile("faces#{}.zip".format(labelID + 1), "w")
      # add multiple images to the zip
      #for current_whole_image in whole_images:
        #zip_obj.write(current_whole_image)
      # close the Zip File
      #zip_obj.close()

    # show the output montage
    current_title = "Face #{}:".format(labelID + 1)
    expander_caption = "Images with Face #{}:".format(labelID + 1)
    current_title = "Unknown:" if labelID == -1 else current_title
    #expander_caption = "Images with Unknown Faces:" if labelID == -1 else expander_caption
    #cv2_imshow(montage)
    with cols1[labelID + 1]:
      st.write(current_title)
      st.image(montage)
    if labelID != -1:
      with st.expander(expander_caption):
        with open("zip_face#{}.zip".format(labelID + 1), "rb") as fp:
          btn = st.download_button(
              label="Download ZIP of Clustered Images with Face#{}".format(labelID + 1),
              data=fp,
              file_name="clustered_faces#{}.zip".format(labelID + 1),
              mime="application/zip"
          )
        fp.close()
        #st.download_button("Download images with Face #{}".format(labelID + 1), data = 'faces#{}.zip'.format(labelID + 1))
        cols2 = st.columns(3)
        for j in range(len(whole_images)):
          with cols2[j%3]:
            st.image(whole_images[j], use_column_width = 'always')        

In [ ]:
!streamlit run streamlit-app.py & npx localtunnel --port 8501